In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pandas as pd
import os
import re
from string import punctuation
from datetime import datetime
import nltk

In [2]:
tidbits = ['.',')','(','/i','&','/a',',', '...', "''", "``", ":'", ";'", '<', 'href=', "''", 'http', ':','//www.j-archive.com/media/2004-12-31_dj_23.mp3', "''", '>']
english_stopwords = set(stopwords.words('english') + list('punctuations') + tidbits)

In [3]:
df = pd.read_json('jeopardy.json')

In [4]:
df

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999


In [26]:
val = df.value.str.strip("$")

In [108]:
print(val)

0          200
1          200
2          200
3          200
4          200
          ... 
216925    2000
216926    2000
216927    2000
216928    2000
216929    None
Name: value, Length: 216930, dtype: object


In [126]:
val.value_counts().head(20)

400      42244
800      31860
200      30455
600      20377
1000     19539
1200     11331
2000     11243
1600     10801
100       9029
500       9016
300       8663
1,000     2101
2,000     1586
3,000      769
1,500      546
1,200      441
4,000      349
1,600      239
2,500      232
5,000      231
Name: value, dtype: int64

In [104]:
len(val)

216930

In [105]:
value = [1 if x is not None and re.search(r'100|200|300|400|500|600|700|800\.', x) else 0 for x in val]


In [106]:
len(value)

216930

In [109]:
print(value)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [6]:
jq = df.question.str.strip("'")

In [8]:
print(jq)

0         For the last 8 years of his life, Galileo was ...
1         No. 2: 1912 Olympian; football star at Carlisl...
2         The city of Yuma in this state has a record av...
3         In 1963, live on "The Art Linkletter Show", th...
4         Signer of the Dec. of Indep., framer of the Co...
                                ...                        
216925    This Puccini opera turns on the solution to 3 ...
216926    In North America this term is properly applied...
216927    In Penny Lane, where this "Hellraiser" grew up...
216928    From Ft. Sill, Okla. he made the plea, Arizona...
216929    A silent movie title includes the last name of...
Name: question, Length: 216930, dtype: object


In [23]:
# Test of concept 
list(zip(jq, val, value))[:30]

[("For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory",
  '200',
  1),
 ('No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves',
  '200',
  1),
 ('The city of Yuma in this state has a record average of 4,055 hours of sunshine each year',
  '200',
  1),
 ('In 1963, live on "The Art Linkletter Show", this company served its billionth burger',
  '200',
  1),
 ('Signer of the Dec. of Indep., framer of the Constitution of Mass., second President of the United States',
  '200',
  1),
 ('In the title of an Aesop fable, this insect shared billing with a grasshopper',
  '200',
  1),
 ("Built in 312 B.C. to link Rome & the South of Italy, it's still in use today",
  '400',
  1),
 ('No. 8: 30 steals for the Birmingham Barons; 2,306 steals for the Bulls',
  '400',
  1),
 ('In the winter of 1971-72, a record 1,122 inches of snow fell at Rainier Paradise Ranger Station in this state',
  '400',
  1),


In [7]:
### To create a cleaned database of words #########################################
def clean_wordlist(qlist):
    corpus = ' '
    lemmatizer = WordNetLemmatizer()
    for i in qlist:
        corpus += '\n'.join([i.lower()])
    wordlist = [lemmatizer.lemmatize(i) for i in word_tokenize(corpus)
                if i not in english_stopwords and not i.startswith('//')]
    return wordlist

In [8]:
cleaned = clean_wordlist(jq)

In [9]:
len(cleaned)

1933020

In [10]:
print(cleaned[:800])

['last', '8', 'year', 'life', 'galileo', 'house', 'arrest', 'espousing', 'man', "'s", 'theoryno', '2', '1912', 'olympian', ';', 'football', 'star', 'carlisle', 'indian', 'school', ';', '6', 'mlb', 'season', 'red', 'giant', 'bravesthe', 'city', 'yuma', 'state', 'record', 'average', '4,055', 'hour', 'sunshine', 'yearin', '1963', 'live', 'art', 'linkletter', 'show', 'company', 'served', 'billionth', 'burgersigner', 'dec.', 'indep.', 'framer', 'constitution', 'mass.', 'second', 'president', 'united', 'statesin', 'title', 'aesop', 'fable', 'insect', 'shared', 'billing', 'grasshopperbuilt', '312', 'b.c', 'link', 'rome', 'south', 'italy', "'s", 'still', 'use', 'todayno', '8', '30', 'steal', 'birmingham', 'baron', ';', '2,306', 'steal', 'bullsin', 'winter', '1971-72', 'record', '1,122', 'inch', 'snow', 'fell', 'rainier', 'paradise', 'ranger', 'station', 'statethis', 'housewares', 'store', 'named', 'packaging', 'merchandise', 'came', 'first', 'displayed', 'away', 'go', 'cow', 'regurgitate', 'fi

In [ ]:
#######################################################################################

In [34]:
allquestions = [ ' '.join(clean_wordlist([x])) for x in df.question.str.strip("'") ]

In [38]:
print(allquestions[:100])

["last 8 year life galileo house arrest espousing man 's theory", '2 1912 olympian ; football star carlisle indian school ; 6 mlb season red giant brave', 'city yuma state record average 4,055 hour sunshine year', '1963 live art linkletter show company served billionth burger', 'signer dec. indep. framer constitution mass. second president united state', 'title aesop fable insect shared billing grasshopper', "built 312 b.c link rome south italy 's still use today", '8 30 steal birmingham baron ; 2,306 steal bull', 'winter 1971-72 record 1,122 inch snow fell rainier paradise ranger station state', 'housewares store named packaging merchandise came first displayed', 'away go', 'cow regurgitate first stomach mouth chew', '1000 rajaraja cholas battled take indian ocean island known tea', "1 lettered hoop football lacrosse syracuse think could n't act ask 11 unclean buddy", "june 28 1994 nat ' l weather service began issuing index rate intensity sun 's radiation", "company 's accutron watch

In [110]:
CAT = df.category

In [90]:
len(value)


216930

In [107]:
len(allquestions)

216930

In [118]:
## I learned my lesson about never using 'values' as a category name within a dataframe - half a day gone trying to
# find out why categories were spilling into eachother! Half a day!!
df2 = pd.DataFrame({'Category': CAT, 'money':value, 'text':allquestions})

In [113]:
df2

,Category,values,text
0,HISTORY,1,last 8 year life galileo house arrest espousin...
1,ESPN's TOP 10 ALL-TIME ATHLETES,1,2 1912 olympian ; football star carlisle india...
2,EVERYBODY TALKS ABOUT IT...,1,"city yuma state record average 4,055 hour suns..."
3,THE COMPANY LINE,1,1963 live art linkletter show company served b...
4,EPITAPHS & TRIBUTES,1,signer dec. indep. framer constitution mass. s...
...,...,...,...
216925,RIDDLE ME THIS,1,puccini opera turn solution 3 riddle posed her...
216926,"""T"" BIRDS",1,north america term properly applied 4 specie c...
216927,AUTHORS IN THEIR YOUTH,1,penny lane hellraiser grew barber shave anothe...
216928,QUOTATIONS,1,ft. sill okla. made plea arizona land home fat...


In [120]:
len(df2.money)

216930

In [115]:
len(df2.text)

216930

In [116]:
print(df2.text)

0         last 8 year life galileo house arrest espousin...
1         2 1912 olympian ; football star carlisle india...
2         city yuma state record average 4,055 hour suns...
3         1963 live art linkletter show company served b...
4         signer dec. indep. framer constitution mass. s...
                                ...                        
216925    puccini opera turn solution 3 riddle posed her...
216926    north america term properly applied 4 specie c...
216927    penny lane hellraiser grew barber shave anothe...
216928    ft. sill okla. made plea arizona land home fat...
216929    silent movie title includes last name 18th c. ...
Name: text, Length: 216930, dtype: object


In [121]:
print(df2.money)

0         1
1         1
2         1
3         1
4         1
         ..
216925    1
216926    1
216927    1
216928    1
216929    0
Name: money, Length: 216930, dtype: int64


In [122]:
x_train, x_test, y_train, y_test = train_test_split(df2.text, df2.money, random_state=1)

In [123]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
x_train_tf = tfidf_vectorizer.fit_transform(x_train)
x_test_tf = tfidf_vectorizer.transform(x_test)

In [124]:
naive_bayes = MultinomialNB()
naive_bayes.fit(x_train_tf, y_train)
predictions = naive_bayes.predict(x_test_tf)

In [125]:
# I'd imagine this accuracy has more to do with the higher frequency of lower dollar amounts than the quality of cleaning I've performed
print('Accuracy: ', accuracy_score(y_test, predictions))

Accuracy:  0.8101340512234249
